In [48]:
import pandas as pd 
from dotenv import load_dotenv
import os
import asyncio  # Nécessaire pour exécuter les requêtes async
from mistralai import Mistral
import json
import time as t

In [49]:
df = pd.read_csv('./src/Cleaned_file.csv', sep=';')
print(df.shape) # (576, 5)
df.head(10) 

(576, 12)


,Nom,Messages,Date_de_publication,Sentiment,Urgence,Probleme,mot-critique,incomfort,type,Deception,Satisfaction,Frustration
0,Guillaume Tournier,@ENGIEpartFR \n6 mois d’attente et tjs aucune ...,2023-11-16 15:13:18+00:00,Négatif,True,"['Attente prolongée pour un devis', ""Absence d...","['6 mois d’attente', 'aucune réponse', 'multip...",80,Service client,85,5,90
1,Jouan Etwan 💚💛,"Bonjour @ENGIEpartSAV , l’appli #monpilotageel...",2023-11-26 07:34:34+00:00,Négatif,True,"[""bug de l'application""]","['bug', 'pas de bol']",80,problème technique,70,0,85
2,thouvenin,@ENGIEpartFR mon syndic de copropriété sergic ...,2023-12-21 14:27:08+00:00,Négatif,True,"[""absence d'eau chaude"", ""non-intervention de ...","['demeure', 'relances', 'scandaleux']",100,service client,100,0,100
3,Anathème Bidule,@ENGIEpartSAV vous envisagez de vous occuper d...,2023-12-28 21:32:58+00:00,Négatif,True,"['Agence fantôme', 'Manque de réponse aux mail...",NaN,90,Service client,80,0,100
4,thouvenin,"@ENGIEpartSAV retour de votre technicien ""vous...",2023-12-29 10:08:10+00:00,Négatif,False,"['problème de pression', 'manque de respect']","['entartrés', 'problème', 'respectez']",80,Service client,85,5,90
5,Zawarudo 😈,@ENGIEpartFR Merci pour l'annulation d'entreti...,2023-12-29 11:13:42+00:00,Négatif,False,"[""annulation d'entretien de dernière minute"", ...","['annulation', 'dernière minute', 'ne comprenn...",80,Service client et annulation d'entretien,85,0,90
6,hawaien88,Je m’explique mal où c’est les personnes du ch...,2023-12-29 12:16:46+00:00,Négatif,True,"['Communication inefficace', 'Incompréhension'...","['comprendre', 'lunaire', 'échange']",80,Service client,70,0,90
7,hawaien88,@ENGIEpartSAV Et c’est si compliqué d’explique...,2023-12-29 14:01:48+00:00,Négatif,False,"[""Manque d'explications"", 'Formation insuffisa...","['expliquer', 'supprimer', 'formés', 'vérifier']",60,Demande d'information,50,10,70
8,Petit,"@ENGIEpartFR Bonjour, J'ai reçu un courrier de...",2023-12-30 13:55:44+00:00,Négatif,True,"[""Délai trop court pour l'installation du comp...","['scandaleux', 'trois jours', ""fêtes de fin d'...",80,Délai d'installation,85,0,90
9,🇷🇸 Ajde Ana! 🇷🇸,@ENGIEpartSAV urgent !! Une honte mon abonneme...,2024-01-01 12:45:32+00:00,Négatif,True,"[""Augmentation inattendue du prix de l'abonnem...","['honte', 'urgent', 'triplé']",100,Facturation,100,0,100


In [50]:
# Sérialisation en JSON
json_data = df[:-176].to_json(orient='records')
json_data_str = json.dumps(json.loads(json_data), indent=0).replace('\n', '')

# Affichage du JSON
print(json_data_str) # 359 150 caractères

[{"Nom": "Guillaume Tournier","Messages": "@ENGIEpartFR \\n6 mois d\u2019attente et tjs aucune r\u00e9ponse de l\u2019agence engie home service grand marseille \u00e0 Aubagne sur un devis, malgr\u00e9 des multiples relances. Je reste coinc\u00e9 \u00e0 appeler la plate-forme t\u00e9l et \u00e0 entendre qu\u2019on me rappelle\\nPour le num\u00e9ro 1 du service client c\u2019est dommage","Date_de_publication": "2023-11-16 15:13:18+00:00","Sentiment": "N\u00e9gatif","Urgence": true,"Probleme": "['Attente prolong\u00e9e pour un devis', \"Absence de r\u00e9ponse de l'agence\", 'Relances infructueuses', 'Service client inefficace']","mot-critique": "['6 mois d\u2019attente', 'aucune r\u00e9ponse', 'multiples relances', 'coinc\u00e9', 'dommage']","incomfort": 80,"type": "Service client","Deception": 85,"Satisfaction": 5,"Frustration": 90},{"Nom": "Jouan Etwan \ud83d\udc9a\ud83d\udc9b","Messages": "Bonjour @ENGIEpartSAV , l\u2019appli #monpilotageelec bug depuis hier, suis-je le seul dans ce c

In [51]:
# Charger les variables d'environnement
load_dotenv()

# Récupérer la clé API
api_key = os.getenv("MISTRAL_API_KEY")
print(f"Clé API récupérée : {api_key}")

# Vérifier que la clé API est bien définie
if not api_key:
    raise ValueError("La clé API MISTRAL_API_KEY n'est pas définie dans l'environnement.")

Clé API récupérée : nnvrycOFgABLo69SLKoL95plqu6N8rQ7


In [52]:
# Initialiser le nouveau client Mistral (asynchrone)
client = Mistral(api_key=api_key)

In [53]:
response =  client.agents.complete(
    agent_id = "ag:753184c9:20250311:untitled-agent:e063ae99",  # Remplace par le bon modèle
    messages = [{"role": "user", "content": json_data_str}]
)

# Transformation de la reponse en json
resp = response.choices[0].message.content.strip()
print(resp)
# Temps : 7m 22.8s

### Liste d'améliorations et préconisations

## Analyse du sentiment général
L'analyse des messages montre un sentiment général négatif avec un taux de satisfaction de 0%, une déception de 100% et une frustration de 90%. Les principaux problèmes identifiés sont liés au service client, à la facturation, aux interventions techniques et à la communication. Les clients expriment une grande insatisfaction concernant la gestion des réclamations, les délais de réponse, le manque de suivi et la mauvaise communication des informations.

## Recommandations et préconisations d'amélioration

1. **Améliorer la Communication et la Transparence :**
   - **Créer un Canal de Communication Dédié :** Mettre en place un canal de communication dédié pour les clients afin de centraliser et de traiter efficacement les réclamations et les demandes d'information.
   - **Mettre à Jour le Site Web et l'Application Mobile :** Assurer que le site web et l'application mobile sont toujours fonctionnels et à jour. Fo

In [54]:
print(response)

id='9e28f7b239d04f3589bc102e55a5c2a5' object='chat.completion' model='mistral-large-2411' usage=UsageInfo(prompt_tokens=114845, completion_tokens=879, total_tokens=115724) created=1741687613 choices=[ChatCompletionChoice(index=0, message=AssistantMessage(content="### Liste d'améliorations et préconisations\n\n## Analyse du sentiment général\nL'analyse des messages montre un sentiment général négatif avec un taux de satisfaction de 0%, une déception de 100% et une frustration de 90%. Les principaux problèmes identifiés sont liés au service client, à la facturation, aux interventions techniques et à la communication. Les clients expriment une grande insatisfaction concernant la gestion des réclamations, les délais de réponse, le manque de suivi et la mauvaise communication des informations.\n\n## Recommandations et préconisations d'amélioration\n\n1. **Améliorer la Communication et la Transparence :**\n   - **Créer un Canal de Communication Dédié :** Mettre en place un canal de communica

In [55]:
print(response.choices[0].message.content)

### Liste d'améliorations et préconisations

## Analyse du sentiment général
L'analyse des messages montre un sentiment général négatif avec un taux de satisfaction de 0%, une déception de 100% et une frustration de 90%. Les principaux problèmes identifiés sont liés au service client, à la facturation, aux interventions techniques et à la communication. Les clients expriment une grande insatisfaction concernant la gestion des réclamations, les délais de réponse, le manque de suivi et la mauvaise communication des informations.

## Recommandations et préconisations d'amélioration

1. **Améliorer la Communication et la Transparence :**
   - **Créer un Canal de Communication Dédié :** Mettre en place un canal de communication dédié pour les clients afin de centraliser et de traiter efficacement les réclamations et les demandes d'information.
   - **Mettre à Jour le Site Web et l'Application Mobile :** Assurer que le site web et l'application mobile sont toujours fonctionnels et à jour. Fo